In [1]:
import math
import string
import time
from collections import defaultdict, deque
from helpful_functions import *
import re
import copy
import functools
import itertools

In [2]:
with open("day22_input.txt", "r") as f:
    prog_inp = f.readlines()

prog_inp = [line.rstrip() for line in prog_inp]

for i in range(len(prog_inp)-1,-1,-1):
    if prog_inp[i] == '': del prog_inp[i]
    else: break
print(prog_inp[:5])

['Player 1:', '13', '25', '18', '6']


In [3]:
iplayer_hands = [deque(), deque()]
current_player_index = 0
for line in prog_inp:
    if line:
        if "Player" in line:
            current_player_index = int(line.split(' ')[-1][:-1]) - 1
        else:
            iplayer_hands[current_player_index].append(int(line))

print(iplayer_hands)

[deque([13, 25, 18, 6, 42, 8, 37, 27, 44, 38, 10, 28, 50, 5, 16, 47, 30, 29, 39, 21, 2, 4, 12, 35, 32]), deque([20, 14, 46, 34, 7, 26, 15, 43, 36, 49, 11, 23, 31, 48, 1, 19, 45, 22, 24, 40, 41, 17, 33, 9, 3])]


In [4]:
def get_score(player_hand):
    score = 0
    for i, card in enumerate(player_hand):
        score += (len(player_hand) - i) * card
    return score

In [5]:
player_hands = copy.deepcopy(iplayer_hands)

game_round = 1
while len(player_hands[0]) > 0 and len(player_hands[1]) > 0:
    cards_in_play = [player_hands[0].popleft(), player_hands[1].popleft()]
    if cards_in_play[0] > cards_in_play[1]:
        player_hands[0].append(cards_in_play[0])
        player_hands[0].append(cards_in_play[1])
    else:
        player_hands[1].append(cards_in_play[1])
        player_hands[1].append(cards_in_play[0])
    game_round += 1

if len(player_hands[0]) > 0:
    winning_player = 0
else:
    winning_player = 1

print(f"Player {winning_player+1} wins!")

score = get_score(player_hands[winning_player])
print(f"Score: {score}")


Player 1 wins!
Score: 34566


In [6]:
def as_hashable(player_hands):
    return (tuple(player_hands[0]), tuple(player_hands[1]),)

def play(player_hands, depth=0, cache=dict()):
    initial_state = as_hashable(player_hands)
    if initial_state in cache:
        # print(depth, player_hands)
        # print("cache hit!")
        return cache[initial_state]

    game_round = 1
    previous_states = set()

    while len(player_hands[0]) > 0 and len(player_hands[1]) > 0:
        ah = as_hashable(player_hands)
        if ah in previous_states:
            player_hands[0].popleft()
            cache[initial_state] = 0
            return 0
        else: previous_states.add(ah)

        cards_in_play = [player_hands[0].popleft(), player_hands[1].popleft()]
        recursive_combat = True
        for i in range(2):
            if len(player_hands[i]) < cards_in_play[i]:
                recursive_combat = False

        if recursive_combat:
            new_deck = []
            for i in range(2):
                new_deck.append(deque())
                j = 0
                for card in player_hands[i]:
                    new_deck[-1].append(card)
                    j += 1
                    if j == cards_in_play[i]:
                        break
                    elif j >= cards_in_play[i]:
                        print('uh oh')
            # itertools.isslice(player_hands[0], 0, cards_in_play[0]), player_hands[1][:cards_in_play[1]]
            round_winner = play(new_deck, depth+1)
        else:
            if cards_in_play[0] > cards_in_play[1]:
                round_winner = 0
            else:
                round_winner = 1

        if round_winner == 0:
            player_hands[0].append(cards_in_play[0])
            player_hands[0].append(cards_in_play[1])
        else:
            player_hands[1].append(cards_in_play[1])
            player_hands[1].append(cards_in_play[0])
        game_round += 1
    

    if len(player_hands[0]) > 0:
        cache[initial_state] = 0
        return 0
    cache[initial_state] = 1
    return 1

player_hands = copy.deepcopy(iplayer_hands)
winner = play(player_hands)

print(f"Player {winner+1} wins!")

score = get_score(player_hands[winner])
print(f"Score: {score}")

Player 1 wins!
Score: 31854
